In [25]:
from SPARQLWrapper import SPARQLWrapper, JSON
from googletrans import Translator
from transliterate import translit
import pandas as pd
import re
import regex as re2
from collections import defaultdict
from time import sleep
import time
from tqdm import tqdm
import wptools
import json
import requests
from urllib.parse import  quote
import mwparserfromhell as mwp
import mwclient
import warnings
warnings.filterwarnings('ignore')
from polyglot.transliteration import Transliterator
from polyglot.text import Text
import pickle

In [2]:
import os, sys
sys.path.append('/home/aniket/')

from ilmulti.translator import from_pretrained
translator = from_pretrained(tag='mm-all-iter1')

| [src] dictionary: 40897 types
| [tgt] dictionary: 40897 types


In [7]:
proxies=['translate.google.com','translate.google.co.kr']
translator = Translator(service_urls=proxies)

In [4]:
class sparqlResults():
    
    def __init__(self,query_file):
        
        self.agent = 'AniketBot (User:Mohanty-Aniket)'
        
        with open(query_file, 'r') as file:
            self.query = file.read()
    
    def _val_cols(self):
        cols = list(self.df.columns.values)
        tmp = []
        for col in cols:
            if(col.split('.')[-1] == 'value'):
                tmp.append(col)
        return tmp
    
    def change_item(self,newItem):
        wikibase_item = newItem
        query_id = '?query_id ' + '{wd:' + wikibase_item + '}'
        self.query = re.sub(r'\?query_id \{.*\}', query_id, self.query)    
    
    def results(self):
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=self.agent)
        sparql.setQuery(self.query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        self.df = pd.json_normalize(results['results']['bindings'])
        return self.df[self._val_cols()]

In [4]:
sparq_obj = sparqlResults('films.rq')
sparq_obj.change_item('Q100801386')

In [68]:
results = sparq_obj.results()

In [34]:
film_cols_dict = {'query_idLabel.value':'name',
                  'image.value':'image',
                  'directorLabel.value':'director',
                  'producer_nameLabel.value':'producer',
                  'screenwriterLabel.value':'screenplay',
                  'based_onnameLabel.value':'based_on', 
                  'castLabel.value':'starring', 
                  'composerLabel.value':'music', 
                  'dopLabel.value':'cinematography', 
                  'editorLabel.value':'editing', 
                  'productioncompanyLabel.value':'studio', 
                  'distributedbyLabel.value':'distributor', 
                  'publication.value':'released', 
                  'durationLabel.value':'runtime', 
                  'origincountryLabel.value':'country', 
                  'original_langLabel.value':'language', 
                  'costLabel.value':'budget', 
                  'boxofficeLabel.value':'gross'}

In [222]:
sports_cols_dict = {
   'query_idLabel.value':'name',
    'image.value':'image',
    'authorityLabel.value':'union',
    'maxplayersLabel.value':'team',
    'gendersLabel.value':'mgender',
    'subclassLabel.value':'type',
    'itemsusedLabel.value':'equipment',
    'origincountryLabel.value':'region'
}

In [6]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
# print(isEnglish('कोलकाता'))

def is_date(text):
    if ('{birth' in text.lower()) or ('{death' in text.lower()):
        return True
    return False

def is_image(text):
    if('.jpg' in text.lower() or '.png' in text.lower()):
        return True
    return False

def Find_URL(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    if "URL" in string:
        check = True
    else:
        check = False
    url = re.findall(regex,string)  
    if url or check:
        return True
    else:
        return False

def indic_translate(eng):
    hi = []
    for i in eng:
        text=quote(i)
        # text=quote('मनिश् जोए')
        url='http://www.cfilt.iitb.ac.in/indicnlpweb/indicnlpws/translate/en/hi/{}/'.format(text)
        ## Note the forward slash '/' at the end of the URL. It's should be there, but please live with it for now!

        response = requests.get(url)
        hi.append(response.json()['hi'])
    return hi

In [7]:
stats = defaultdict(list)

for i in range(1457):
    
    with open('./stats/' + 'instance_' + str(i) + '.pkl', 'rb') as f:
        data = pickle.load(f)
    
    for key in data:
        stats[key].extend(data[key])

with open('./updated_parsed_data.pkl','rb') as f:
    data = pickle.load(f)

dict_tmp = {}
for key in data: #data from parsed_data.pkl
    dict_tmp[data[key]['wikibase_item']] = data[key]['infobox']

for key in stats:
    for idx,item in enumerate(stats[key]):
        wikibase_item = item['item']
        if(wikibase_item in dict_tmp):
            stats[key][idx]['infobox'] = dict_tmp[wikibase_item]

In [8]:
total_count = 0
noinfo_count = 0
films_items_noinfobox = []
films_with_infobox = []
print('Instances considered :- ')
for key in stats:
    if('film' in key[-4:]):
        print(key)
        for val in stats[key]:
            if(val['infobox'] == 0):
                films_items_noinfobox.append(val['item'])
                noinfo_count += 1
            else:
                films_with_infobox.append(val['item'])
        total_count += len(stats[key])
print('\nTotal film pages: ',total_count)
print('Film no infobox: ',noinfo_count)

print('\n\n-------\n')

total_count = 0
noinfo_count = 0
sports_items_noinfobox = []
print('Instances considered :- ')
for key in stats:
    if('sport' in key[-5:] and 'transport' not in key):
        print(key)
        for val in stats[key]:
            if(val['infobox'] == 0):
                sports_items_noinfobox.append(val['item'])
                noinfo_count += 1
        total_count += len(stats[key])
print('\nTotal sports pages: ',total_count)
print('Sports no infobox: ',noinfo_count)

Instances considered :- 
film
short film
animated feature film
television film
feature film
silent short film
animated film
3D film
cult film
anthology film

Total film pages:  4872
Film no infobox:  1507


-------

Instances considered :- 
type of sport
sport
dog sport
fictional sport
individual sport
winter sport
mind sport

Total sports pages:  63
Sports no infobox:  45


In [224]:
def create_infobox_from_wikidata(query_file, wikibase_item, translate = False):
    
    try:
        sparq_obj = sparqlResults(query_file)
        sparq_obj.change_item(wikibase_item)

        results = sparq_obj.results()
    except:
        return []

    infobox = []
    print(results.columns.values)
    for col in sports_cols_dict:
        if(col in results.columns.values):
            col_name = sports_cols_dict[col]
            val = pd.unique(results[col])
            hindi = []  
            if(translate):
                if(col_name == 'released'):
                    val = [val[0][:-10]]
                for vals in val:
                    if isEnglish(vals) and not Find_URL(vals):
                        translated = translator(vals, tgt_lang='hi', src_lang='en')
                        translated = translated[0]['tgt']
                        check = True
        #                 if isEnglish(translated):
        #                     check = False
                        if check:
                            hindi = hindi + [translated]
                    else:
                        if not Find_URL(vals):
                            try:
                                vals = translit(vals,reversed = True)
                                translated = translator(vals, tgt_lang='hi', src_lang='en')
                                translated = translated[0]['tgt']
                                check = True
        #                         if isEnglish(translated):
        #                             check = False
                                if check:
                                    hindi = hindi + [translated]
                            except:
                                hindi.append(vals)
                        else:
                            hindi.append(vals)
                info_str = col_name + ' = ' + ', '.join(hindi)
            else:
                hindi.append(', '.join(val))
                info_str = col_name + ' = ' + hindi[0]

            infobox.append(info_str)
    return infobox

In [213]:
def create_all_film_infobox(item_list, translate = False):
    film_infoboxes = {}
    errs = []
    if(translate):
        file = './sports.rq'
    else:
        file = './films_eng.rq'
    for item in tqdm(item_list):
        res = create_infobox_from_wikidata(file, item, translate)
        if(res != []):
            film_infoboxes[item] = res
        else:
            errs.append(item)
    return film_infoboxes, errs

In [225]:
film_infoboxes,film_errs = create_all_film_infobox(['Q1734'], translate=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['image.value' 'icon.value' 'pronounciation.value' 'query_idLabel.value'
 'subclassLabel.value' 'practicedbyLabel.value' 'itemsusedLabel.value'
 'authorityLabel.value' 'minplayersLabel.value' 'maxplayersLabel.value'
 'instanceLabel.value' 'gendersLabel.value']


100%|██████████| 1/1 [00:02<00:00,  2.84s/it]


In [226]:
film_infoboxes

{'Q1734': ['name = वालीबॉल',
  'image = http://commons.wikimedia.org/wiki/Special:FilePath/Red%20dragons%20kwalificeren%20zich%20tegen%20grieken%20voor%20ek.jpg',
  'union = फेडरेशन इंटरनेशनल डे वोलीबॉल',
  'team = 12',
  'mgender = महिलाओं का वॉलीबॉल, पुरुषों की वॉलीबॉल',
  'type = टीम खेल, गेंद खेल',
  'equipment = वॉलीबॉल, वॉलीबॉल कोर्ट']}

In [108]:
with open('film_infoboxes_english_wikidata.pkl', 'wb') as handle:
    pickle.dump(film_infoboxes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open('film_infoboxes.pkl', 'wb') as handle:
    pickle.dump(film_infoboxes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [90]:
entity_id = 'Q170564'
titles = wptools.page(wikibase = entity_id, silent = True).get_wikidata()
title_list = titles.data['title'].split('_')
title = " ".join(title_list)
page = wptools.page(title, silent = True)
page = page.get_parse()
page.data['infobox']

{'name': 'Terminator 2: Judgment Day',
 'image': 'Terminator2poster.jpg',
 'caption': 'Theatrical release poster',
 'director': '[[James Cameron]]',
 'producer': 'James Cameron',
 'writer': '{{Plainlist|\n* James Cameron\n* [[William Wisher]]}}',
 'starring': '{{Plainlist|\n* [[Arnold Schwarzenegger]]\n* [[Linda Hamilton]]\n* [[Robert Patrick]]\n|<!--These three actors are the only ones listed on the billing block; per [[Template:Infobox film]]. -->}}',
 'music': '[[Brad Fiedel]]',
 'cinematography': '[[Adam Greenberg (cinematographer)|Adam Greenberg]]',
 'editing': '{{Plainlist|\n* [[Conrad Buff]]\n* [[Mark Goldblatt]]\n* [[Richard A. Harris]]}}',
 'studio': '{{Plainlist|\n* [[Carolco Pictures]]\n* [[Pacific Western Productions]]\n* [[Lightstorm Entertainment]]\n* [[StudioCanal|Le Studio Canal+ S.A.]]}}',
 'distributor': '[[TriStar Pictures]]',
 'released': '{{Film date|1991|7|1|Los Angeles|1991|7|3|United States}}',
 'runtime': '137 minutes',
 'country': 'United States',
 'language':

In [11]:
def create_infobox_from_english_wiki(page, translate = False):
    key_en = list(page.data['infobox'].keys())
    value_en = list(page.data['infobox'].values())
    infobox = []

    for ind in range(len(key_en)):
        flag = 0
#         if('flatlist' in value_en[ind]):
        if Find_URL(value_en[ind]):
            continue

        value_en[ind] = value_en[ind].strip('{}')
        value_en[ind] = re.sub(r'flatlist','',value_en[ind])
        value_en[ind] = re.sub(r'Unbulleted list','',value_en[ind])
        value_en[ind] = re.sub(r'Plainlist','',value_en[ind])
        value_en[ind] = value_en[ind].strip('*')
#         print(value_en[ind])
        value_en[ind] = re.sub(r'\|',' ',value_en[ind])
        value_en[ind] = re.sub('[^0-9a-zA-Z ,.\.,.-]+','',value_en[ind])
#         print(value_en[ind])
        if not Find_URL(value_en[ind]) and not is_date(value_en[ind]):
#             print(value_en[ind])
            vals = mwp.parse(value_en[ind]).strip_code().replace('*','')
        
        vals = value_en[ind].strip()
        hindi = []
        if(translate and vals):
            if(key_en[ind] in ['image', 'imagesize']):
                infobox.extend([key_en[ind]+" = "+value_en[ind]])
                continue
            if not is_date(vals) and not is_image(vals):
                if isEnglish(vals) and not Find_URL(vals):
                    translated = translator(vals, tgt_lang='hi', src_lang='en')
#                     print(translated,'val: ',vals)
                    translated = translated[0]['tgt']
                    hindi.append(translated)
                else:
                    if not Find_URL(vals):
                        try:
                            vals = translit(vals,reversed = True)
                            translated = translator(vals, tgt_lang='hi', src_lang='en')
                            translated = translated[0]['tgt']
                            hindi.append(translated)
                        except:
                            hindi.append(vals)
                    else:
                        hindi.append(vals)
            else:
                hindi.append(vals)
        else:
            hindi.append(vals)
        infobox.extend([key_en[ind]+" = "+hindi[0]])
    return infobox

In [12]:
def create_all_film_from_english(item_list, translate = False):
    no_infobox = []
    errs = []
    infoboxes = {}
    cnt = 0
    for entity_id in tqdm(item_list):
        try:
            titles = wptools.page(wikibase = entity_id, silent = True).get_wikidata()
            title_list = titles.data['title'].split('_')
            title = " ".join(title_list)
            page = wptools.page(title, silent = True)
            page = page.get_parse()
            if(page.data['infobox']):
    #             print('infobox',page.data['infobox'])
                infoboxes[entity_id] = create_infobox_from_english_wiki(page, translate)
                cnt += 1
            else:
                no_infobox.append(entity_id)
        except:
            errs.append(entity_id)
        print(cnt)
    return infoboxes, no_infobox, errs

In [202]:
film_infoboxes_from_english, film_no_english_infobox, film_errs_from_english = create_all_film_from_english(['Q1734'], translate = True)

100%|██████████| 1/1 [00:13<00:00, 13.21s/it]

1


In [14]:
# with open('film_infoboxes_gt_translated.pkl', 'wb') as handle:
#     pickle.dump(film_infoboxes_from_english, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
def create_infobox_string(infobox_list,template):
#     template = 'writer'
    infobox_str = '{{Infobox ' + template + '\n'
    for line in infobox_list:
        infobox_str += '|' + line + '\n'
    infobox_str += '}}'
    return infobox_str

In [68]:
def publish_to_sandbox(infobox_str,input_page,credentials):
    with open(credentials,'r') as cred:
        for line in cred:
            if(line[:4] == 'user'): user = line.split('=')[-1].strip('\n')
            if(line[:8] == 'password'): password = line.split('=')[-1]
    site = mwclient.Site('hi.wikipedia.org')
    site.login(user, password)
    old_page = site.Pages[input_page]
    old_text = old_page.text()
    old_text = re2.sub(r'(?=\{Infobox)(\{([^{}]|(?1))*\})','',old_text,flags = re.S)
    old_text = re2.sub(r'(?=\{ज्ञानसन्दूक)(\{([^{}]|(?1))*\})','',old_text,flags = re.S)
#     print(old_page.text())
    page = site.Pages['User:Mohanty-Aniket/प्रयोगपृष्ठ']
#     page = site.Pages['User:Mohanty-Aniket/sandbox']
    text = infobox_str
    page.save(text + '\n' + old_text,summary='trying wikidata-infobox')

In [227]:
infobox_str = create_infobox_string(film_infoboxes['Q1734'],'film')

In [48]:
# publish_to_sandbox(infobox_str,'Baahubali:_The_Beginning','credentials.txt')

In [228]:
print(infobox_str)

{{Infobox film
|name = वालीबॉल
|image = http://commons.wikimedia.org/wiki/Special:FilePath/Red%20dragons%20kwalificeren%20zich%20tegen%20grieken%20voor%20ek.jpg
|union = फेडरेशन इंटरनेशनल डे वोलीबॉल
|team = 12
|mgender = महिलाओं का वॉलीबॉल, पुरुषों की वॉलीबॉल
|type = टीम खेल, गेंद खेल
|equipment = वॉलीबॉल, वॉलीबॉल कोर्ट
}}


In [51]:
with open('film_infoboxes.pkl', 'rb') as handle:
    wikidat_info = pickle.load(handle)

In [53]:
with open('film_infoboxes_gt_translated.pkl', 'rb') as handle:
    trans_info = pickle.load(handle)

In [54]:
valid_pages = [list(wikidat_info.keys()),list(trans_info.keys())]
res = set(valid_pages[0]).intersection(*valid_pages)
valid_pages = list(res)

In [56]:
valid_pages[:5]

['Q190908', 'Q23647131', 'Q62023603', 'Q47703', 'Q3765055']

In [198]:
for i in [0]:
    title = trans_info[valid_pages[i]][0].split('=')[-1].strip()
#     print(title)
#     if(title == 'द मार्शियन'):
    title = '_'.join(title.split())
    print(title)
    infobox_str = create_infobox_string(trans_info[valid_pages[i]],'film')
    print(infobox_str)
#         print(i)

सात
{{Infobox film
|name = सात
|image = Seven movie poster.jpg
|caption = थियेटर रिलीज पोस्टर
|director = डेविड फंचर
|producers = जुब्ल अरनल्ड कोपेल्सन फिलिस कार्लाइल
|writer = एंड्रयू केविन वॉकर
|starring = ubl Brad Pitt Morgan riman Gwyneth Paltro John McGinley ------------
|music = हावर्ड शॉयर
|cinematography = डेरियस खोंडजी
|editing = रिचर्ड फ्रांसिस-ब्रूस
|studio = जुब्ल चेकी गोरी चित्र जूनो पिक्स
|distributor = नई पंक्ति सिनेमा
|released = फिल्म की तारीख 1995 09 15 एलिस टुली हॉल 1995 09 22 अमेरिका
|runtime = 127 मिनट
|country = अमेरिका
|language = अंग्रेजी
|budget = 33 करोड़
|gross = 327.3 करोड़
}}


In [229]:
publish_to_sandbox(infobox_str,'वालीबॉल','credentials.txt')